<a href="https://colab.research.google.com/github/yoshihikom/thu_Details_of_Management_Information_BII_Lesson12/blob/main/hara2021_chap6_13_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.13 テキスト分析(4) トピックモデル(LDA)
トピックモデルでは、ある特定の要素について、語句が構成する割合を示すことが可能になる。

今回は、DBpedia.orgから取得した社会学者のデータからモデルを作成します。

In [1]:
!pip install SPARQLWrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.3 MB/s eta 0:00:00


In [2]:
from SPARQLWrapper import SPARQLWrapper
import codecs

#
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#日本の経済学者の情報をまとめて取得する
sparql2 = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')
sparql2.setQuery("""
PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>

select distinct ?name ?abstract where {
    ?company <http://dbpedia.org/ontology/wikiPageWikiLink> <http://ja.dbpedia.org/resource/Category:日本の社会学者> .
    ?company rdfs:label ?name .
    ?company <http://dbpedia.org/ontology/abstract> ?abstract .
}
""")
results = []
results2 = sparql2.query().convert()

# このままresult2をprilltするとjson形式のデータで出力されるため、nameおよび bstractのvalueのみ取り出すことにしましょう。
# 結果を整形
for result in results2["results"]["bindings"]:
#    print("経済",";",result["name"]["value"],";",result["abstract"]["value"])
    results.append(result["abstract"]["value"])
# これにより,Wikipedia(DBPedia.org)に掲載された経済学者の名前およびその概要に関する情報を一括して取得することが出来ました
#このリストをtextに変換。リスト要素の区切りは改行とする。
text = '\n'.join(results)

f = codecs.open('社会学者リスト_dbpedia.txt', 'w', 'utf8')
f.write(text)
f.close()

- janome.tokenizerからTokennizerパッケージをインポートする
- テキストデータを取り込み分かち書き処理をする
- wvsに分かち書きしたベクトルを一行ずつ代入していく

In [3]:
!pip install janome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 70.2 MB/s eta 0:00:00


In [4]:
import codecs as cd
import gensim
from janome.tokenizer import Tokenizer
from gensim import corpora, models, similarities

filename = '社会学者リスト_dbpedia.txt'
file = cd.open(filename,  'r', 'utf-8')
lines = file.readlines()

t = Tokenizer()
wvs = []

for i, line in enumerate(lines):
    # 社会学者ごとに語のベクトルを作成する
    word_vector = []

    # 短すぎる場合は無視
    if len(line)<30:
        continue
    # 記号以外はベクトル作成
    else:
        tokens = t.tokenize(line)

        for token in tokens:
            if token.part_of_speech[:2] == '名詞':
                word_vector += [token.base_form]

    # データを連結
        wvs += [word_vector]

In [5]:
print(wvs)

[['ゃんた', '1969', '年', '7', '月', '18', '日', '-', 'スリランカ', '生まれ', '日本', '社会', '学者', 'タレント', '羽衣', '国際', '大学', '現代', '社会学部', '教授', '学位', '博士', '経済', '学', '講演', '家', '落語', '家', '活動', '初代', '京都', '府', '名誉', '友好', '大使', 'セイ', 'ロン', 'スリランカ', 'キャンディ', '出生', '名', '\u200d', 'Jayasinghe', 'Arachilage', 'Thusitha', 'Devapriya', 'Nishantha', 'ジャヤシンハ・アーラッチラーゲー・トシタ・デーワップリヤ・ニシャンタ', 'ゃんたとは', '静か', '夜', 'こと', 'シン', 'ハラ', '語', '静夜', '意味', '2019', '年', '7', '月', '時点', 'プロフィール', '長年', '静夜', '意味', '名前', '2018', '年', '秋', '頃', '夜', '意味', 'Nishaa', '意味', 'Anta', '混成', '語', 'つまり', 'ゃんたとは', '夜明け', '意味', '追記'], ['しこ', 'ひで', 'のり', '益子', '英', '雅', '1960', '年', '10', '月', '-', '2021', '年', '9', '月', '日本', '社会', '学者', '茨城', '県', '出身', '中京大学', '国際', '教養', '学部', '教授', '日本', '解放', '社会', '学会', '理事', '法政大学', '沖縄', '文化', '研究所', '国内', '研究', '員', '沖縄大学', '民俗', '研究所', '特別', '研究', '員'], ['アンジェロ・アキミツ・イシ', 'Angelo', 'Akimitsu', 'Ishi', '1967', '年', '-', '日本', '拠点', 'ブラジル', '国籍', '社会', '学者', '武蔵大学', '教授', '日系', 'ブラジル', '人', '3',

In [6]:
# 辞書作成
dictionary = corpora.Dictionary(wvs)
dictionary.filter_extremes(no_below=2, no_above=0.3)
dictionary.save_as_text('dict_sociology.txt')

# コーパスを作成
corpus = [dictionary.doc2bow(text) for text in wvs]
corpora.MmCorpus.serialize('cop_sociology.mm', corpus)

In [7]:
dictionary = gensim.corpora.Dictionary.load_from_text('dict_sociology.txt')
corpus = corpora.MmCorpus('cop_sociology.mm')

In [8]:
# gensim.models.ldamodek.load_from_text('')
topic_number = 15
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=topic_number, id2word=dictionary)

for i in range(topic_number):
    print('トピック:', i, '__', lda.print_topic(i))

トピック: 0 __ 0.024*"論" + 0.014*"文化" + 0.013*"大学院" + 0.012*"福祉" + 0.012*"者" + 0.011*"科" + 0.009*"博士" + 0.009*"国際" + 0.009*"宗教" + 0.008*"東京大学"
トピック: 1 __ 0.030*"大学" + 0.014*"博士" + 0.014*"大学院" + 0.013*"政策" + 0.011*"出身" + 0.008*"科学" + 0.008*"講師" + 0.008*"理事" + 0.008*"11" + 0.008*"長"
トピック: 2 __ 0.018*"情報" + 0.016*"1" + 0.015*"大学" + 0.010*"専攻" + 0.010*"2" + 0.010*"学科" + 0.009*"理事" + 0.009*"10" + 0.008*"東京" + 0.008*"博士"
トピック: 3 __ 0.022*"教育" + 0.013*"博士" + 0.012*"国際" + 0.011*"大学" + 0.011*"論" + 0.009*"委員" + 0.009*"5" + 0.009*"3" + 0.009*"会" + 0.009*"科"
トピック: 4 __ 0.033*"論" + 0.015*"博士" + 0.014*"科" + 0.014*"大学院" + 0.013*"科学" + 0.010*"文化" + 0.010*"委員" + 0.010*"大学" + 0.009*"学会" + 0.009*"文学部"
トピック: 5 __ 0.014*"東京大学" + 0.014*"大学院" + 0.012*"者" + 0.012*"大学" + 0.012*"論" + 0.010*"情報" + 0.009*"7" + 0.009*"学部" + 0.009*"3" + 0.009*"家"
トピック: 6 __ 0.015*"博士" + 0.013*"者" + 0.012*"県" + 0.012*"大学" + 0.011*"出身" + 0.011*"大学院" + 0.010*"論" + 0.009*"東京大学" + 0.008*"助教授" + 0.008*"文化"
トピック: 7 __ 0.015*"科" + 0.013*"文学" +